<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_7/blob/main/Session7_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import sys
import pandas as pd
import numpy as np
import csv

In [32]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [33]:
sentences = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Session5/stanfordSentimentTreebank/datasetSentences.txt',sep='\t')
sentences.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [34]:
sentences.shape

(11855, 2)

In [35]:
sentiment = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Session5/stanfordSentimentTreebank/sentiment_labels.txt',sep='|')
sentiment.head()

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [36]:
sentiment_class = []
for i in sentiment['sentiment values'] :
  if i >=0 and i<0.2:
    sentiment_class.append(1)
  elif i>=0.2 and i<0.4:
    sentiment_class.append(2)
  elif i>=0.4 and i<0.6:
    sentiment_class.append(3)
  elif i>=0.6 and i<0.8:
    sentiment_class.append(4)
  else:
    sentiment_class.append(5)

sentiment['sentiment values'] = sentiment_class


In [37]:
sentiment.describe()

,phrase ids,sentiment values
count,239232.000000,239232.000000
mean,119615.500000,3.062391
std,69060.474137,0.911298
min,0.000000,1.000000
25%,59807.750000,3.000000
50%,119615.500000,3.000000
75%,179423.250000,4.000000
max,239231.000000,5.000000


In [38]:
dataset = pd.merge(left=sentences, right=sentiment, left_on='sentence_index', right_on='phrase ids')
dataset = dataset.drop(columns=['phrase ids','sentence_index'])
dataset.rename(columns = {'sentiment values':'sentiments'}, inplace = True)
print(dataset.shape)
print(dataset.head())
print(dataset.columns)


(11855, 2)
                                            sentence  sentiments
0  The Rock is destined to be the 21st Century 's...           3
1  The gorgeously elaborate continuation of `` Th...           3
2                     Effective but too-tepid biopic           3
3  If you sometimes like to go to the movies to h...           3
4  Emerges as something rare , an issue movie tha...           2
Index(['sentence', 'sentiments'], dtype='object')


In [50]:
def smallCase(data):
  for i in data.index:
    data['sentence'][i] = data['sentence'][i].lower()
  return data

In [52]:
import re
def cleanText(data):
  data_small_case = smallCase(data)
  for i in data_small_case.index:
    data_small_case.sentence[i] = re.sub("[^-9A-Za-z ]", "" , data_small_case.sentence[i])

  return data_small_case

dataset = cleanText(dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [53]:
dataset.head(10)

,sentence,sentiments
0,the rock is destined to be the st century s ne...,3
1,the gorgeously elaborate continuation of the ...,3
2,effective but too-tepid biopic,3
3,if you sometimes like to go to the movies to h...,3
4,emerges as something rare an issue movie that...,2
5,the film provides some great insight into the ...,3
6,offers that rare combination of entertainment ...,3
7,perhaps no picture ever made has more literall...,2
8,steers turns in a snappy screenplay that curls...,3
9,but he somehow pulls it off,3


In [54]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.3)
train = train.reset_index().drop(columns=['index'])
test = test.reset_index().drop(columns=['index'])

In [55]:
print(train.shape)
print(test.shape)

(8298, 2)
(3557, 2)


In [76]:
test.sentence[1:10]

1                      one-of-a-kind near-masterpiece 
2    for movie lovers as well as opera lovers  tosc...
3    excellent performances from jacqueline bisset ...
4     a story  an old and scary one  about the mons...
5    a painfully slow cliche-ridden film filled wit...
6        a smart  arch and rather cold-blooded comedy 
7    it does nt reach them  but the effort is grate...
8    cremaster  is at once a tough pill to swallow ...
9    shadyac shoots his film like an m night shyama...
Name: sentence, dtype: object

In [56]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [57]:
sentence = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
sentiments = torchtext.legacy.data.Field(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [58]:
fields = [('sentence', sentence),('sentiments',sentiments)]

In [59]:
example = [torchtext.legacy.data.Example.fromlist([train.sentence[i],train.sentiments[i]], fields) for i in range(train.shape[0])] 

In [61]:
trainDataset = torchtext.legacy.data.Dataset(example, fields)

In [62]:
example = [torchtext.legacy.data.Example.fromlist([test.sentence[i],test.sentiments[i]], fields) for i in range(test.shape[0])] 

In [63]:
testDataset = torchtext.legacy.data.Dataset(example, fields)

In [64]:
vars(trainDataset.examples[10])

{'sentence': ['the',
  'plot',
  'is',
  'romantic',
  'comedy',
  'boilerplate',
  'from',
  'start',
  'to',
  'finish'],
 'sentiments': 3}

In [65]:
sentence.build_vocab(trainDataset)
sentiments.build_vocab(trainDataset)

In [66]:
print('Size of input vocab : ', len(sentence.vocab))
print('Size of label vocab : ', len(sentiments.vocab))
print('Top 10 words appreared repeatedly :', list(sentence.vocab.freqs.most_common(10)))
print('Labels : ', sentiments.vocab.stoi)

Size of input vocab :  15090
Size of label vocab :  6
Top 10 words appreared repeatedly : [(' ', 8846), ('the', 7113), ('a', 5066), ('and', 4441), ('of', 4296), ('to', 2998), ('-', 2641), ('s', 2542), ('is', 2466), ('it', 2374)]
Labels :  defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f299d70b810>>, {'<unk>': 0, 3: 1, 2: 2, 4: 3, 5: 4, 1: 5})


In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [68]:
train_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits((trainDataset, testDataset), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [69]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(sentence.vocab.stoi, tokens)

In [70]:
# Define hyperparameters
size_of_vocab = len(sentence.vocab)
embedding_dim = 200
num_hidden_nodes = 300
num_output_nodes = 6
num_layers = 4
dropout = 0.4

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [71]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(15090, 200)
  (encoder): LSTM(200, 300, num_layers=4, batch_first=True, dropout=0.4)
  (fc): Linear(in_features=300, out_features=6, bias=True)
)
The model has 5,789,406 trainable parameters


In [72]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [73]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentence  
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        # compute the loss
        loss = criterion(predictions, batch.sentiments)        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.sentiments)   
        # backpropage the loss and compute the gradients
        loss.backward()       
        # update the weights
        optimizer.step()      
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [74]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, sentence_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(sentence, sentence_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.sentiments)
            acc = binary_accuracy(predictions, batch.sentiments)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [75]:
N_EPOCHS = 10
best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    
    # save the best model
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Test. Loss: {test_loss:.3f} |  Test. Acc: {test_acc*100:.2f}% \n')

	Train Loss: 1.560 | Train Acc: 52.21%
	 Test. Loss: 1.534 |  Test. Acc: 51.27% 

	Train Loss: 1.515 | Train Acc: 53.12%
	 Test. Loss: 1.534 |  Test. Acc: 51.29% 

	Train Loss: 1.515 | Train Acc: 53.10%
	 Test. Loss: 1.534 |  Test. Acc: 51.27% 

	Train Loss: 1.513 | Train Acc: 53.12%
	 Test. Loss: 1.534 |  Test. Acc: 51.24% 

	Train Loss: 1.512 | Train Acc: 53.34%
	 Test. Loss: 1.533 |  Test. Acc: 51.24% 

	Train Loss: 1.510 | Train Acc: 53.60%
	 Test. Loss: 1.534 |  Test. Acc: 51.16% 

	Train Loss: 1.509 | Train Acc: 53.93%
	 Test. Loss: 1.533 |  Test. Acc: 51.27% 

	Train Loss: 1.506 | Train Acc: 54.16%
	 Test. Loss: 1.535 |  Test. Acc: 50.93% 

	Train Loss: 1.503 | Train Acc: 54.45%
	 Test. Loss: 1.533 |  Test. Acc: 51.02% 

	Train Loss: 1.501 | Train Acc: 54.57%
	 Test. Loss: 1.536 |  Test. Acc: 50.82% 



In [80]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentence(sentence):
    
    #categories = {0: "very negative", 1:"negative", 2:"neutral", 3:"positive", 4:"very positive"}
    #'<unk>': 0, 3: 1, 2: 2, 4: 3, 5: 4, 1: 5
    categories = {0: "unknown", 1:"neutral", 2:"negative", 3:"positive", 4:"very positive", 5:"very negative"}
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [84]:
for i in range(10):
  print(test.sentence[i])
  print("predicted: ", classify_sentence(test.sentence[i]))
  print("actual: ", test.sentiments[i])

bon apptit 
predicted:  neutral
actual:  3
one-of-a-kind near-masterpiece 
predicted:  neutral
actual:  3
for movie lovers as well as opera lovers  tosca is a real treat 
predicted:  neutral
actual:  2
excellent performances from jacqueline bisset and martha plimpton grace this deeply touching melodrama 
predicted:  neutral
actual:  3
 a story  an old and scary one  about the monsters we make  and the vengeance they take 
predicted:  neutral
actual:  2
a painfully slow cliche-ridden film filled with more holes than clyde barrow s car 
predicted:  neutral
actual:  2
a smart  arch and rather cold-blooded comedy 
predicted:  neutral
actual:  3
it does nt reach them  but the effort is gratefully received 
predicted:  neutral
actual:  4
cremaster  is at once a tough pill to swallow and a minor miracle of self-expression 
predicted:  neutral
actual:  3
shadyac shoots his film like an m night shyamalan movie  and he frequently maintains the same snail s pace  he just forgot to add any genuine